In [1]:
import json # read from json file
import re
from collections import Counter 
from sklearn.utils import shuffle

In [2]:
### RUN
with open('annotations2.jsonl') as jsonl_file:
    # note: after running data-preprocessing.ipynb this file already has token-level labels
    lines = jsonl_file.readlines()
annot = [json.loads(line) for line in lines]
#print(annot[0])

In [3]:
### RUN
# now convert annotation tokens into list (sentences) of lists (tokens) format for sklearn_crfsuite.CRF
train_sents=[] 
for j in range(0,len(annot)):
    a = annot[j]['tokens']
    train_sentence = []
    for i in range(0,len(a)):
        if 'label' in a[i]: # only add element if this sample sentence has been labelled 
            token_element = (a[i]['text'],a[i]['label'])
            train_sentence.append(token_element)
    train_sents.append(train_sentence)

In [7]:
labels = sorted({a for seq in train_sents for _,a in seq})

In [9]:
# summary counts
# first unfold y_train
y_train_all = [a for seq in train_sents for _,a in seq]
y_train_all_count = Counter(y_train_all)
print(y_train_all_count)
print(len(y_train_all))

Counter({'O': 5600, 'ORT': 238, 'DATUM_VERTRAG': 222, 'IMMO_TYP': 219, 'VERKAEUFER': 215, 'DATUM_VERBUECHERUNG': 212, 'STRASSE': 166, 'KAEUFER': 153, 'FLAECHE': 139, 'GESAMTPREIS': 103, 'QMPREIS': 77, 'TERRASSENGROESSE': 36})
7380


In [10]:
# extract counters for all annot 
for l in range(0,len(labels)):
    out=[]
    for k in range(0,len(annot)):
        j=len(annot[k]['tokens'])
        for i in range(0,j):
            if annot[k]['spans']!=[]: # are there annot for this example?
                if annot[k]['tokens'][i]['label']==labels[l]:
                    out.append(annot[k]['tokens'][i]['text'].lower())
    out_count = Counter(out)
    print(out_count)

Counter({'2021': 55, '2020': 42, 'oktober': 18, 'dezember': 14, 'juli': 13, 'august': 12, 'september': 9, 'november': 9, 'februar': 8, 'april': 7, 'juni': 7, 'mai': 6, 'mit': 4, 'jänner': 3, '2022': 2, 'märz': 2, '2019': 1})
Counter({'september': 14, 'august': 12, 'juli': 12, 'november': 12, 'mai': 11, 'oktober': 10, 'april': 10, '6.': 9, 'juni': 8, '12.': 7, '3.': 7, '18.': 6, '1.': 6, '29.': 6, '2.': 6, 'dezember': 6, '4.': 5, '27.': 5, '5.': 5, '23.': 5, '16.': 4, 'märz': 4, 'jänner': 4, '7.': 3, '10.': 3, 'am': 3, 'februar': 3, '2019': 3, '28.': 3, '20.': 3, '25.': 3, '31.': 3, '2020': 3, '19.': 2, '15.': 2, '30.': 2, '24.': 2, '21.': 2, '11.': 1, '9.': 1, '25.04.2019': 1, '8.': 1, '13.': 1, '2012': 1, '14.': 1, '26.': 1})
Counter({'75,14': 2, '1526': 2, '82': 2, '145': 2, '142': 2, '71,93': 1, '100,67': 1, '76,9': 1, '35': 1, '197': 1, '63,25': 1, '781': 1, '98,84': 1, '1827': 1, '54,32': 1, '66,92': 1, '104,38': 1, '78,86': 1, '52,43': 1, '90,88': 1, '104': 1, '422': 1, '46,5': 1

In [11]:
# extract # of tokens for each annot 
out=Counter({})
for k in range(0,len(annot)):
    j=len(annot[k]['tokens']) #74
    i=0
    if annot[k]['spans']!=[]: # are there annot for this example?
        while i < j:
            m=0 # count length of successive tokens w/ same label
            #print("k/"+str(k))
            #print("i/"+str(i))
            while (i < (j-1)) and annot[k]['tokens'][i]['label']==annot[k]['tokens'][i+1]['label']:
                 # note w/ i=j-2 in below expression can go up to i+1 = j-1 ... i.e. i can go up to 72
                m+=1
                i+=1
                #print(annot[k]['tokens'][i]['label'])
                #print(m)
            m+=1
            out += Counter({(annot[k]['tokens'][i]['label'],m): 1}) # add 1 entry for this pair of label and length
            #if (annot[k]['tokens'][i]['label'] == 'GESAMTPREIS') and (m>1): # investigating situations where # successive tokens GESAMTPREIS > 1
            #    print("k/"+str(k))
            #    print("i/"+str(i))
            i+=1
            #print(out)
                   # out.append(annot[k]['tokens'][i]['text'].lower())
print(out)

Counter({('O', 1): 313, ('O', 2): 276, ('ORT', 1): 238, ('O', 5): 151, ('IMMO_TYP', 1): 138, ('FLAECHE', 1): 137, ('O', 7): 136, ('DATUM_VERTRAG', 2): 97, ('GESAMTPREIS', 1): 96, ('DATUM_VERBUECHERUNG', 2): 96, ('O', 4): 95, ('QMPREIS', 1): 75, ('O', 8): 59, ('VERKAEUFER', 1): 55, ('STRASSE', 1): 53, ('O', 6): 52, ('STRASSE', 2): 39, ('KAEUFER', 1): 38, ('O', 3): 38, ('TERRASSENGROESSE', 1): 36, ('O', 10): 19, ('O', 13): 19, ('KAEUFER', 2): 17, ('O', 12): 17, ('O', 9): 14, ('IMMO_TYP', 4): 14, ('VERKAEUFER', 2): 13, ('O', 11): 12, ('VERKAEUFER', 3): 11, ('KAEUFER', 3): 11, ('O', 15): 10, ('STRASSE', 3): 9, ('DATUM_VERTRAG', 3): 9, ('O', 14): 9, ('DATUM_VERBUECHERUNG', 1): 8, ('VERKAEUFER', 5): 7, ('KAEUFER', 4): 7, ('KAEUFER', 5): 4, ('DATUM_VERBUECHERUNG', 3): 4, ('VERKAEUFER', 4): 4, ('VERKAEUFER', 8): 3, ('IMMO_TYP', 5): 3, ('O', 18): 3, ('VERKAEUFER', 6): 2, ('STRASSE', 4): 2, ('GESAMTPREIS', 2): 2, ('IMMO_TYP', 2): 2, ('VERKAEUFER', 7): 2, ('O', 16): 2, ('IMMO_TYP', 3): 2, ('O', 2

In [12]:
# Statistics of dataset
sumExamples = len(annot)
sumTokens = 0
sumLabels = 0
sumLabelledWords = 0
labelWords =[]
for i in range (0,sumExamples):
    sumTokens+=len(annot[i]['tokens'])
    noLabels = len(annot[i]['spans'])
    sumLabels += noLabels
    for j in range(0,noLabels):
        labelWords.append((annot[i]['spans'][j]['label'],annot[i]['spans'][j]['noWords'])) 
        sumLabelledWords+=annot[i]['spans'][j]['noWords']
print("Total Examples = " + str(sumExamples))
print("Total Distinct Labels = " + str(sumLabels))
print("Total Distinct Labelled Tokens = " + str(sumLabelledWords))
print("Avg. # of words per label = "+str(sumLabelledWords/sumLabels))
print("Avg. # of words per example = "+str(sumTokens/sumExamples))
print("Avg. # of labelled words per example = "+str(sumLabelledWords/sumExamples))
print("Avg. # of labels per example = "+str(sumLabels/sumExamples))
labelWordDistrib = Counter(labelWords)
labelWordDistrib_items = labelWordDistrib.items()
labelWordDistrib_sorted = sorted(labelWordDistrib_items)
print(labelWordDistrib_sorted)

Total Examples = 140
Total Distinct Labels = 1283
Total Distinct Labelled Tokens = 1822
Avg. # of words per example = 55.121428571428574
Avg. # of labelled words per example = 13.014285714285714
Avg. # of labels per example = 9.164285714285715
Avg. # of words per label = 1.4201091192517536
[(('DATUM_VERBUECHERUNG', 1), 8), (('DATUM_VERBUECHERUNG', 2), 96), (('DATUM_VERBUECHERUNG', 3), 4), (('DATUM_VERTRAG', 1), 1), (('DATUM_VERTRAG', 2), 97), (('DATUM_VERTRAG', 3), 9), (('FLAECHE', 1), 138), (('GESAMTPREIS', 1), 97), (('GESAMTPREIS', 2), 3), (('IMMO_TYP', 1), 142), (('IMMO_TYP', 2), 2), (('IMMO_TYP', 3), 2), (('IMMO_TYP', 4), 14), (('IMMO_TYP', 5), 3), (('KAEUFER', 1), 69), (('KAEUFER', 2), 17), (('KAEUFER', 3), 11), (('KAEUFER', 4), 7), (('KAEUFER', 5), 4), (('ORT', 1), 238), (('QMPREIS', 1), 76), (('STRASSE', 1), 53), (('STRASSE', 2), 39), (('STRASSE', 3), 9), (('STRASSE', 4), 2), (('TERRASSENGROESSE', 1), 36), (('VERKAEUFER', 1), 64), (('VERKAEUFER', 2), 13), (('VERKAEUFER', 3), 11)

In [20]:
#Average number of words per label for each different label type
a = [((a1,a2),a3) for ((a1,a2),a3) in labelWordDistrib.items()]

keys = [a1 for ((a1,a2),a3) in a]

values1 = [a3 for ((a1,a2),a3) in a]

values2 = [a2*a3 for ((a1,a2),a3) in a]

d = {}
e = {}
for item in range(len(keys)):
    key = keys[item]
    value1 = values1[item]
    value2 = values2[item]
    if key in d:
        d[key] += value1
        e[key] += value2
    else:
        d[key] = value1
        e[key] = value2
f={}
for key in d:
    f[key]=e[key]/d[key]
print(f)

{'ORT': 1.0, 'STRASSE': 1.6116504854368932, 'FLAECHE': 1.0, 'IMMO_TYP': 1.3680981595092025, 'QMPREIS': 1.0, 'TERRASSENGROESSE': 1.0, 'KAEUFER': 1.7037037037037037, 'VERKAEUFER': 2.1132075471698113, 'GESAMTPREIS': 1.03, 'DATUM_VERTRAG': 2.074766355140187, 'DATUM_VERBUECHERUNG': 1.962962962962963}


In [21]:
# lexicon
a=[]
for i in range(0,len(annot)):
    for j in range(0,len(annot[i]['tokens'])):
        a.append(annot[i]['tokens'][j]['text'].lower())
b=set(a)
print(len(a)) # total # of words
print(len(b)) # total # of distinct words

7717
1152


In [22]:
# length of each example
sumExamples = len(annot)
for i in range (0,sumExamples):
    if annot[i]['spans']!=[]:
        print("i is:"+str(i)+";"+"length is: "+str(len(annot[i]['tokens'])))

i is:0;length is: 77
i is:1;length is: 73
i is:2;length is: 67
i is:3;length is: 55
i is:4;length is: 71
i is:5;length is: 67
i is:6;length is: 66
i is:7;length is: 68
i is:8;length is: 57
i is:9;length is: 70
i is:10;length is: 67
i is:11;length is: 63
i is:12;length is: 59
i is:13;length is: 66
i is:14;length is: 71
i is:15;length is: 53
i is:16;length is: 72
i is:17;length is: 66
i is:18;length is: 57
i is:19;length is: 57
i is:20;length is: 54
i is:21;length is: 61
i is:22;length is: 77
i is:23;length is: 65
i is:24;length is: 52
i is:25;length is: 78
i is:26;length is: 55
i is:27;length is: 61
i is:28;length is: 53
i is:29;length is: 62
i is:30;length is: 65
i is:31;length is: 51
i is:32;length is: 68
i is:33;length is: 70
i is:34;length is: 59
i is:35;length is: 72
i is:36;length is: 56
i is:37;length is: 53
i is:38;length is: 61
i is:39;length is: 64
i is:40;length is: 66
i is:41;length is: 73
i is:42;length is: 71
i is:43;length is: 65
i is:44;length is: 70
i is:45;length is: 7

In [23]:
# Token counts per class label
auxCount = Counter([a for seq in train_sents for _,a in seq])
print(auxCount)


Counter({'O': 5600, 'ORT': 238, 'DATUM_VERTRAG': 222, 'IMMO_TYP': 219, 'VERKAEUFER': 215, 'DATUM_VERBUECHERUNG': 212, 'STRASSE': 166, 'KAEUFER': 153, 'FLAECHE': 139, 'GESAMTPREIS': 103, 'QMPREIS': 77, 'TERRASSENGROESSE': 36})
